In [1]:
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\Jurgen\Desktop\FYP\Shared\News_Twitter_Stock_Reddit_NaN_Final.csv")
df = df.drop(["Date", "Stockname", "Close_USD"], axis=1)

df

,News_Senti,Volume,Open_USD,High_USD,Low_USD,Adj_Close_USD,Twitter_Senti,Reddit_Comment,Reddit_SelfText
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.167460,0.352600
1,NaN,13112253.0,41.508241,42.088464,40.615591,39.984037,NaN,0.091721,-0.167850
2,-0.013258,10959655.0,41.112864,41.741913,41.112864,40.782627,NaN,0.195632,0.065820
3,NaN,8776763.0,41.783077,42.096227,41.201514,40.692290,NaN,0.089100,0.192950
4,NaN,7039773.0,41.233912,41.636413,40.786690,40.416246,NaN,0.107269,0.191075
...,...,...,...,...,...,...,...,...,...
483,NaN,NaN,NaN,NaN,NaN,NaN,0.6369,NaN,NaN
484,0.107508,6431522.0,43.335973,44.019775,43.335973,43.848824,0.6369,NaN,NaN
485,0.139891,7581629.0,44.008072,44.264928,43.494359,43.751215,0.6729,NaN,NaN
486,NaN,8959435.0,43.406557,43.492172,42.978485,43.149714,0.9357,NaN,NaN


# Non NaN Stock Data + Sentiments Filled With 0

In [3]:
df[["News_Senti","Twitter_Senti", "Reddit_Comment", "Reddit_SelfText"]] = df[["News_Senti","Twitter_Senti", "Reddit_Comment", "Reddit_SelfText"]].fillna(0)
df = df.dropna().reset_index(drop=True)
print(df)

X = df.loc[:, df.columns != 'Adj_Close_USD'].to_numpy()
y = df["Adj_Close_USD"].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=0)

print(X_train.shape[0])
print(X_test.shape[0])


     News_Senti      Volume   Open_USD   High_USD    Low_USD  Adj_Close_USD  \
0      0.000000  13112253.0  41.508241  42.088464  40.615591      39.984037   
1     -0.013258  10959655.0  41.112864  41.741913  41.112864      40.782627   
2      0.000000   8776763.0  41.783077  42.096227  41.201514      40.692290   
3      0.000000   7039773.0  41.233912  41.636413  40.786690      40.416246   
4      0.000000  17843706.0  40.692186  41.005202  39.932002      39.487791   
..          ...         ...        ...        ...        ...            ...   
318    0.000000   7879897.0  43.240752  43.581230  42.985393      43.325872   
319    0.107508   6431522.0  43.335973  44.019775  43.335973      43.848824   
320    0.139891   7581629.0  44.008072  44.264928  43.494359      43.751215   
321    0.000000   8959435.0  43.406557  43.492172  42.978485      43.149714   
322    0.107235  10304777.0  43.661609  44.004053  43.062332      43.147943   

     Twitter_Senti  Reddit_Comment  Reddit_SelfText

# Dropping All Rows With NaN

In [ ]:
df_dropped = df.dropna().reset_index(drop=True)
df_dropped.shape


X = df_dropped.loc[:, df_dropped.columns != 'Adj_Close_USD'].to_numpy()
y = df_dropped["Adj_Close_USD"].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

print(X_train.shape[0])
print(X_test.shape[0])
df_dropped

# Random Forest Regressor

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

r2_value = []
for i in range(100, 1100, 100):
    text_classifier = RandomForestRegressor(n_estimators=i, random_state=0)
    text_classifier.fit(X_train, y_train)
    y_pred = text_classifier.predict(X_test)
    r2_value.append((metrics.r2_score(y_test, y_pred),i))
    print("R^2: {:.8f} n_estimators = {}".format(metrics.r2_score(y_test, y_pred), i))

r2 = pd.DataFrame(r2_value, columns = ["R2", "n_estimators"]).set_index("n_estimators")
n_est = r2.R2[r2.R2 == r2.R2.max()].index[0]


text_classifier = RandomForestRegressor(n_estimators = n_est, random_state=0)
text_classifier.fit(X_train, y_train)
y_pred = text_classifier.predict(X_test)

print()
print("N_Estimators: ", n_est)
print("R^2: ", metrics.r2_score(y_test, y_pred))
print("Variance Score: ", metrics.explained_variance_score(y_test, y_pred))
print("Max Error: ", metrics.max_error(y_test, y_pred))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R^2: 0.96681532 n_estimators = 100
R^2: 0.96685539 n_estimators = 200
R^2: 0.96601133 n_estimators = 300
R^2: 0.96601565 n_estimators = 400
R^2: 0.96594055 n_estimators = 500
R^2: 0.96607593 n_estimators = 600
R^2: 0.96614013 n_estimators = 700
R^2: 0.96625053 n_estimators = 800
R^2: 0.96622733 n_estimators = 900
R^2: 0.96627378 n_estimators = 1000

N_Estimators:  200
R^2:  0.9668553888106292
Variance Score:  0.9669585338788049
Max Error:  0.9745191507949826
Mean Absolute Error: 0.29357407443183
Mean Squared Error: 0.13100500185680672
Root Mean Squared Error: 0.36194613115325147


In [5]:
import matplotlib.pyplot as plt

# Plot outputs

plt.plot(y_test, color = "red")
plt.plot(y_pred, color = "green")


plt.xticks(())
plt.yticks(())

plt.show()

<Figure size 640x480 with 1 Axes>

In [ ]:
test = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Difference': (y_pred - y_test)})
test

# Linear Regression

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.5f'
      % r2_score(y_test, y_pred))
# The intercept
print("Intercept: ", regr.intercept_)

# K Nearest Neighbours

In [ ]:
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt

rmse_val = [] #to store rmse values for different k
for K in range(20):
    K = K+1
    model = neighbors.KNeighborsRegressor(n_neighbors = K)

    model.fit(X_train, y_train)  #fit the model
    pred=model.predict(X_test) #make prediction on test set
    error = sqrt(mean_squared_error(y_test, pred)) #calculate rmse
    rmse_val.append(error) #store rmse values
    
rmse_val = pd.DataFrame(rmse_val)
print(rmse_val)
print(rmse)

In [ ]:
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt
import numpy as np

rmse_val = [] #to store rmse values for different k
for K in range(7):
    K = K+1
    model = neighbors.KNeighborsRegressor(n_neighbors = K)

    model.fit(X_train, y_train)  #fit the model
    pred=model.predict(X_test) #make prediction on test set
    error = sqrt(mean_squared_error(y_test, pred)) #calculate rmse
    rmse_val.append(error) #store rmse values
    
rmse_val = pd.DataFrame(rmse_val)
# print(rmse_val)
print("K = ",rmse_val.idxmin()[0])

In [ ]:
from sklearn import metrics
from sklearn import neighbors
from math import sqrt




knn = neighbors.KNeighborsRegressor(n_neighbors = 6)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)


print("R^2: ", metrics.r2_score(y_test, y_pred))
print("Variance Score: ", metrics.explained_variance_score(y_test, y_pred))
print("Max Error: ", metrics.max_error(y_test, y_pred))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


# CLASSIFIER

In [ ]:
#Classifier

# df1 = pd.read_csv(r"C:\Users\Jurgen\Desktop\FYP\News\SK-Learn\News_Stock_Data_30.csv")
# X = df1["News_Senti"].to_numpy().reshape(-1, 1)
# y = df1["Stock_Changes"].to_numpy()
# df1

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
count = 0
for i in range(len(y_test)):
    if y_test[i] == y_pred[i]:
        count += 1
print(count, '/', len(y_test))
print(y_test)
print(y_pred)